<a href="https://colab.research.google.com/github/ReidelVichot/LC_identification/blob/main/census_cleaning_11_18_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Problem Definition

Background

There seems to be a relationship between the agglomeration of logistical activiey and air pollution. To assess this relationship, the author will use a difference in difference natural experiment design with a synthetic control group. The level of analysis is US contigous counties from 1998 to 2022.

Problem

To conduct a synthetic control group analysis, we need to have a set of covariates and controls to construct this synthetic control group. Using information from the US census, we can construct a set of variables that includes covariates and control variables for the analysis.
I will get variables for each county and each year, including total population, share of white, share of male, age groups, industries, and time of commuting.

#Data Collection

In [68]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
# -- import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [70]:
# -- set directory path
dpath = "/content/drive/MyDrive/Disertation/Census/"
fname = "DECENNIALDPSF42000.DP1-Data.csv"
# -- create dataframes
decennial_00 = pd.read_csv(dpath + fname, skiprows=1)
# -- add year variable
decennial_00["year"] = 2000

#Data Cleaning

##Decennial 2000

In [71]:
# -- create GEOID
decennial_00["GEOID"] = decennial_00.Geography.str[-5:]

# -- select necesary columns
cols = ['GEOID','Geography', 'Geographic Area Name', 'Race/Ethnic Group',
       'Population Groups', 'Number!!Total population',
       'Number!!Total population!!SEX AND AGE!!Male',
       'Number!!Total population!!SEX AND AGE!!Female',
       'Number!!Total population!!SEX AND AGE!!Under 5 years',
       'Number!!Total population!!SEX AND AGE!!5 to 9 years',
       'Number!!Total population!!SEX AND AGE!!10 to 14 years',
       'Number!!Total population!!SEX AND AGE!!15 to 19 years',
       'Number!!Total population!!SEX AND AGE!!20 to 24 years',
       'Number!!Total population!!SEX AND AGE!!25 to 34 years',
       'Number!!Total population!!SEX AND AGE!!35 to 44 years',
       'Number!!Total population!!SEX AND AGE!!45 to 54 years',
       'Number!!Total population!!SEX AND AGE!!55 to 59 years',
       'Number!!Total population!!SEX AND AGE!!60 to 64 years',
       'Number!!Total population!!SEX AND AGE!!65 to 74 years',
       'Number!!Total population!!SEX AND AGE!!75 to 84 years',
       'Number!!Total population!!SEX AND AGE!!85 years and over',
       'Number!!Total population!!SEX AND AGE!!Median age (years)',
       'Number!!HOUSEHOLDS BY TYPE!!Households',
       'Number!!HOUSEHOLDS BY TYPE!!Households!!Average household size',
       'year']

decennial_00 = decennial_00[cols]

In [72]:
# -- rename columns for simplicity
renamed_columns = {'Number!!Total population': "total_population",
                   'Number!!Total population!!SEX AND AGE!!Male': "male",
                   'Number!!Total population!!SEX AND AGE!!Female': "female",
                   'Number!!Total population!!SEX AND AGE!!Under 5 years': "under_5_years",
                   'Number!!Total population!!SEX AND AGE!!5 to 9 years': '5_to_9_years',
                   'Number!!Total population!!SEX AND AGE!!10 to 14 years': '10_to_14_years',
                   'Number!!Total population!!SEX AND AGE!!15 to 19 years': '15_to_19_years',
                   'Number!!Total population!!SEX AND AGE!!20 to 24 years': '20_to_24_years',
                   'Number!!Total population!!SEX AND AGE!!25 to 34 years': '25_to_34_years',
                   'Number!!Total population!!SEX AND AGE!!35 to 44 years': '35_to_44_years',
                   'Number!!Total population!!SEX AND AGE!!45 to 54 years': '45_to_54_years',
                   'Number!!Total population!!SEX AND AGE!!55 to 59 years': '55_to_59_years',
                   'Number!!Total population!!SEX AND AGE!!60 to 64 years': '60_to_64_years',
                   'Number!!Total population!!SEX AND AGE!!65 to 74 years': '65_to_74_years',
                   'Number!!Total population!!SEX AND AGE!!75 to 84 years': '75_to_84_years',
                   'Number!!Total population!!SEX AND AGE!!85 years and over': '85_years_and_over',
                   'Number!!Total population!!SEX AND AGE!!Median age (years)': 'median_age',
                   'Number!!HOUSEHOLDS BY TYPE!!Households': 'households',
                   'Number!!HOUSEHOLDS BY TYPE!!Households!!Average household size': 'average_household_size'}
decennial_00.rename(columns=renamed_columns, inplace=True)

# -- create age groups
decennial_00["Age < 15"] = decennial_00['under_5_years'] + decennial_00['5_to_9_years'] + decennial_00['10_to_14_years']
decennial_00["Age 15-24"] = decennial_00['15_to_19_years'] + decennial_00['20_to_24_years']
decennial_00["Age 25-44"] = decennial_00['25_to_34_years'] + decennial_00['35_to_44_years']
decennial_00["Age 45-64"] = decennial_00['45_to_54_years'] + decennial_00['55_to_59_years'] + decennial_00['60_to_64_years']
decennial_00["Age >= 65"] = decennial_00['65_to_74_years'] + decennial_00['75_to_84_years'] + decennial_00['85_years_and_over']

# -- drop unnecesary columns
decennial_00.drop(columns= ['under_5_years', '5_to_9_years', '10_to_14_years',
                            '15_to_19_years', '20_to_24_years', '25_to_34_years',
                            '35_to_44_years', '45_to_54_years', '55_to_59_years',
                            '60_to_64_years', '65_to_74_years', '75_to_84_years',
                            '85_years_and_over'], inplace=True)

# -- create a variable for white population
white = decennial_00[decennial_00["Race/Ethnic Group"] == 2][["total_population"]].values.copy()

# -- add variable to the decennial dataframe
decennial_00 = decennial_00[decennial_00['Race/Ethnic Group'] == 1].reset_index(drop=True)
decennial_00["white"] = white
del white

decennial_00.drop(columns=['Geography', 'Geographic Area Name',
                           'Race/Ethnic Group', 'Population Groups'], inplace=True)

# -- organize columns
cols = ['GEOID', 'year', 'total_population', 'white', 'male', 'female', 'median_age',
       'households', 'average_household_size', 'Age < 15', 'Age 15-24',
       'Age 25-44', 'Age 45-64', 'Age >= 65']
decennial_00 = decennial_00[cols]


##ACS 2010

In [73]:
fname = "ACSST5Y2010.S0101-Data.csv"
acs5_10 = pd.read_csv(dpath + fname, skiprows=1)

In [74]:
# -- create GEOID
acs5_10["GEOID"] = acs5_10.Geography.str[-5:]
# -- create year
acs5_10["year"] = 2010

# -- drop columns
cols = ['GEOID', 'year',
        'Total!!Estimate!!Total population',
        'Male!!Estimate!!Total population',
        'Female!!Estimate!!Total population',
        'Total!!Estimate!!AGE!!Under 5 years',
        'Total!!Estimate!!AGE!!5 to 9 years',
        'Total!!Estimate!!AGE!!10 to 14 years',
        'Total!!Estimate!!AGE!!15 to 19 years',
        'Total!!Estimate!!AGE!!20 to 24 years',
        'Total!!Estimate!!AGE!!25 to 29 years',
        'Total!!Estimate!!AGE!!30 to 34 years',
        'Total!!Estimate!!AGE!!35 to 39 years',
        'Total!!Estimate!!AGE!!40 to 44 years',
        'Total!!Estimate!!AGE!!45 to 49 years',
        'Total!!Estimate!!AGE!!50 to 54 years',
        'Total!!Estimate!!AGE!!55 to 59 years',
        'Total!!Estimate!!AGE!!60 to 64 years',
        'Total!!Estimate!!AGE!!65 to 69 years',
        'Total!!Estimate!!AGE!!70 to 74 years',
        'Total!!Estimate!!AGE!!75 to 79 years',
        'Total!!Estimate!!AGE!!80 to 84 years',
        'Total!!Estimate!!AGE!!85 years and over',
        'Total!!Estimate!!SUMMARY INDICATORS!!Median age (years)']

acs5_10 = acs5_10[cols]
# -- rename columns for simplicity
renamed_columns = {'Total!!Estimate!!Total population': "total_population",
                   'Male!!Estimate!!Total population': "male",
                   'Female!!Estimate!!Total population': "female",
                   'Total!!Estimate!!AGE!!Under 5 years': "under_5_years",
                   'Total!!Estimate!!AGE!!5 to 9 years': '5_to_9_years',
                   'Total!!Estimate!!AGE!!10 to 14 years': '10_to_14_years',
                   'Total!!Estimate!!AGE!!15 to 19 years': '15_to_19_years',
                   'Total!!Estimate!!AGE!!20 to 24 years': '20_to_24_years',
                   'Total!!Estimate!!AGE!!25 to 29 years': '25_to_29_years',
                   'Total!!Estimate!!AGE!!30 to 34 years': '30_to_34_years',
                   'Total!!Estimate!!AGE!!35 to 39 years': '35_to_39_years',
                   'Total!!Estimate!!AGE!!40 to 44 years': '40_to_44_years',
                   'Total!!Estimate!!AGE!!45 to 49 years': '45_to_49_years',
                   'Total!!Estimate!!AGE!!50 to 54 years': '50_to_54_years',
                   'Total!!Estimate!!AGE!!55 to 59 years': '55_to_59_years',
                   'Total!!Estimate!!AGE!!60 to 64 years': '60_to_64_years',
                   'Total!!Estimate!!AGE!!65 to 69 years': '65_to_69_years',
                   'Total!!Estimate!!AGE!!70 to 74 years': '70_to_74_years',
                   'Total!!Estimate!!AGE!!75 to 79 years': '75_to_79_years',
                   'Total!!Estimate!!AGE!!80 to 84 years': '80_to_84_years',
                   'Total!!Estimate!!AGE!!85 years and over': '85_years_and_over',
                   'Total!!Estimate!!SUMMARY INDICATORS!!Median age (years)': 'median_age'}
acs5_10 = acs5_10.rename(columns=renamed_columns).copy()

# -- create age groups
acs5_10["Age < 15"] = acs5_10['under_5_years'] + acs5_10['5_to_9_years'] + acs5_10['10_to_14_years']
acs5_10["Age 15-24"] = acs5_10['15_to_19_years'] + acs5_10['20_to_24_years']
acs5_10["Age 25-44"] = acs5_10['25_to_29_years'] + acs5_10['30_to_34_years'] + acs5_10['35_to_39_years'] + acs5_10['40_to_44_years']
acs5_10["Age 45-64"] = acs5_10['45_to_49_years'] + acs5_10['50_to_54_years'] + acs5_10['55_to_59_years'] + acs5_10['60_to_64_years']
acs5_10["Age >= 65"] = acs5_10['65_to_69_years'] + acs5_10['70_to_74_years'] + acs5_10['75_to_79_years'] + acs5_10['80_to_84_years'] + acs5_10['85_years_and_over']


# -- drop unnecesary columns
acs5_10.drop(columns=['under_5_years', '5_to_9_years', '10_to_14_years',
                      '15_to_19_years', '20_to_24_years', '25_to_29_years',
                      '30_to_34_years', '35_to_39_years', '40_to_44_years',
                      '45_to_49_years', '50_to_54_years', '55_to_59_years',
                      '60_to_64_years', '65_to_69_years', '70_to_74_years',
                      '75_to_79_years', '80_to_84_years', '85_years_and_over'], inplace=True)

In [75]:
acs5_10_dp05 = pd.read_csv(dpath + "ACSDP5Y2010.DP05-Data.csv", skiprows=1, low_memory=False)

In [76]:
acs5_10.head()

,GEOID,year,total_population,male,female,median_age,Age < 15,Age 15-24,Age 25-44,Age 45-64,Age >= 65
0,01001,2010,53155,25780,27375,36.2,22.1,14.1,27.4,24.9,11.5
1,01003,2010,175791,85902,89889,41.0,19.3,11.6,24.6,28.1,16.5
2,01005,2010,27699,14652,13047,38.0,18.4,13.4,27.3,27.0,13.9
3,01007,2010,22610,12162,10448,38.3,19.1,14.3,28.1,26.2,12.5
4,01009,2010,56692,28080,28612,38.3,20.5,12.4,26.5,26.5,14.3


In [77]:
# -- add white variable from ACS DP05
acs5_10_dp05 = pd.read_csv(dpath + "ACSDP5Y2010.DP05-Data.csv", skiprows=1, low_memory=False)
acs5_10_dp05 = acs5_10_dp05[['Geography',"Estimate!!RACE!!White", "Estimate!!Total housing units"]]
acs5_10_dp05['GEOID'] = acs5_10_dp05.Geography.str[-5:]
acs5_10_dp05.drop(columns=['Geography'], inplace=True)
acs5_10_dp05.rename(columns={'Estimate!!RACE!!White': 'white', "Estimate!!Total housing units":'households'}, inplace=True)

acs5_10 = acs5_10.merge(acs5_10_dp05, on='GEOID').copy()
del acs5_10_dp05
acs5_10.head()



,GEOID,year,total_population,male,female,median_age,Age < 15,Age 15-24,Age 25-44,Age 45-64,Age >= 65,white,households
0,01001,2010,53155,25780,27375,36.2,22.1,14.1,27.4,24.9,11.5,42758,21530
1,01003,2010,175791,85902,89889,41.0,19.3,11.6,24.6,28.1,16.5,153434,101093
2,01005,2010,27699,14652,13047,38.0,18.4,13.4,27.3,27.0,13.9,13972,12011
3,01007,2010,22610,12162,10448,38.3,19.1,14.3,28.1,26.2,12.5,19054,8885
4,01009,2010,56692,28080,28612,38.3,20.5,12.4,26.5,26.5,14.3,54543,23482


##ACS 2011

In [78]:
acs5_11 = pd.read_csv(dpath + "ACSDP5Y2011.DP05-Data.csv", skiprows=1, low_memory=False)
# -- create GEOID
acs5_11["GEOID"] = acs5_11.Geography.str[-5:]
# -- create year
acs5_11["year"] = 2011

# -- select columns
cols = ['GEOID', 'year',
         'Estimate!!RACE!!White',
         'Estimate!!SEX AND AGE!!Under 5 years',
         'Estimate!!SEX AND AGE!!5 to 9 years',
         'Estimate!!SEX AND AGE!!10 to 14 years',
         'Estimate!!SEX AND AGE!!15 to 19 years',
         'Estimate!!SEX AND AGE!!20 to 24 years',
         'Estimate!!SEX AND AGE!!25 to 34 years',
         'Estimate!!SEX AND AGE!!35 to 44 years',
         'Estimate!!SEX AND AGE!!45 to 54 years',
         'Estimate!!SEX AND AGE!!55 to 59 years',
         'Estimate!!SEX AND AGE!!60 to 64 years',
         'Estimate!!SEX AND AGE!!65 years and over',
         'Estimate!!SEX AND AGE!!Female',
         'Estimate!!SEX AND AGE!!Male',
         'Estimate!!SEX AND AGE!!Median age (years)',
         'Estimate!!SEX AND AGE!!Total population',
         'Estimate!!Total housing units']
acs5_11 = acs5_11[cols]


In [79]:
renamed_columns = {'Estimate!!RACE!!White': 'white',
         'Estimate!!SEX AND AGE!!Under 5 years': "under_5_years" ,
         'Estimate!!SEX AND AGE!!5 to 9 years': '5_to_9_years',
         'Estimate!!SEX AND AGE!!10 to 14 years': '10_to_14_years',
         'Estimate!!SEX AND AGE!!15 to 19 years': '15_to_19_years',
         'Estimate!!SEX AND AGE!!20 to 24 years': '20_to_24_years',
         'Estimate!!SEX AND AGE!!25 to 34 years': '25_to_34_years',
         'Estimate!!SEX AND AGE!!35 to 44 years': '35_to_44_years',
         'Estimate!!SEX AND AGE!!45 to 54 years': '45_to_54_years',
         'Estimate!!SEX AND AGE!!55 to 59 years': '55_to_59_years',
         'Estimate!!SEX AND AGE!!60 to 64 years': '60_to_64_years',
         'Estimate!!SEX AND AGE!!65 years and over': '65_years_and_over',
         'Estimate!!SEX AND AGE!!Female': 'female',
         'Estimate!!SEX AND AGE!!Male': 'male',
         'Estimate!!SEX AND AGE!!Median age (years)': 'median_age',
         'Estimate!!SEX AND AGE!!Total population': 'total_population',
         'Estimate!!Total housing units': 'households'}
acs5_11.rename(columns=renamed_columns, inplace=True)

# -- create age groups
acs5_11["Age < 15"] = acs5_11['under_5_years'] + acs5_11['5_to_9_years'] + acs5_11['10_to_14_years']
acs5_11["Age 15-24"] = acs5_11['15_to_19_years'] + acs5_11['20_to_24_years']
acs5_11["Age 25-44"] = acs5_11['25_to_34_years'] + acs5_11['35_to_44_years']
acs5_11["Age 45-64"] = acs5_11['45_to_54_years'] + acs5_11['55_to_59_years'] + acs5_11['60_to_64_years']
acs5_11["Age >= 65"] = acs5_11['65_years_and_over']


# -- drop unnecesary columns
acs5_11.drop(columns=['under_5_years', '5_to_9_years', '10_to_14_years',
                      '15_to_19_years', '20_to_24_years', '25_to_34_years',
                      '35_to_44_years', '45_to_54_years', '55_to_59_years',
                      '60_to_64_years', '65_years_and_over'], inplace=True)



##ACS 2012

In [80]:
acs5_12 = pd.read_csv(dpath + "ACSDP5Y2012.DP05-Data.csv", skiprows=1, low_memory=False)

# -- create GEOID
acs5_12["GEOID"] = acs5_12.Geography.str[-5:]
# -- create year
acs5_12["year"] = 2012

# -- select columns
cols = ['GEOID', 'year',
         'Estimate!!RACE!!White',
         'Estimate!!SEX AND AGE!!Under 5 years',
         'Estimate!!SEX AND AGE!!5 to 9 years',
         'Estimate!!SEX AND AGE!!10 to 14 years',
         'Estimate!!SEX AND AGE!!15 to 19 years',
         'Estimate!!SEX AND AGE!!20 to 24 years',
         'Estimate!!SEX AND AGE!!25 to 34 years',
         'Estimate!!SEX AND AGE!!35 to 44 years',
         'Estimate!!SEX AND AGE!!45 to 54 years',
         'Estimate!!SEX AND AGE!!55 to 59 years',
         'Estimate!!SEX AND AGE!!60 to 64 years',
         'Estimate!!SEX AND AGE!!65 years and over',
         'Estimate!!SEX AND AGE!!Female',
         'Estimate!!SEX AND AGE!!Male',
         'Estimate!!SEX AND AGE!!Median age (years)',
         'Estimate!!SEX AND AGE!!Total population',
         'Estimate!!Total housing units']
acs5_12 = acs5_12[cols]

renamed_columns = {'Estimate!!RACE!!White': 'white',
         'Estimate!!SEX AND AGE!!Under 5 years': "under_5_years" ,
         'Estimate!!SEX AND AGE!!5 to 9 years': '5_to_9_years',
         'Estimate!!SEX AND AGE!!10 to 14 years': '10_to_14_years',
         'Estimate!!SEX AND AGE!!15 to 19 years': '15_to_19_years',
         'Estimate!!SEX AND AGE!!20 to 24 years': '20_to_24_years',
         'Estimate!!SEX AND AGE!!25 to 34 years': '25_to_34_years',
         'Estimate!!SEX AND AGE!!35 to 44 years': '35_to_44_years',
         'Estimate!!SEX AND AGE!!45 to 54 years': '45_to_54_years',
         'Estimate!!SEX AND AGE!!55 to 59 years': '55_to_59_years',
         'Estimate!!SEX AND AGE!!60 to 64 years': '60_to_64_years',
         'Estimate!!SEX AND AGE!!65 years and over': '65_years_and_over',
         'Estimate!!SEX AND AGE!!Female': 'female',
         'Estimate!!SEX AND AGE!!Male': 'male',
         'Estimate!!SEX AND AGE!!Median age (years)': 'median_age',
         'Estimate!!SEX AND AGE!!Total population': 'total_population',
         'Estimate!!Total housing units': 'households'}
acs5_12.rename(columns=renamed_columns, inplace=True)

# -- create age groups
acs5_12["Age < 15"] = acs5_12['under_5_years'] + acs5_12['5_to_9_years'] + acs5_12['10_to_14_years']
acs5_12["Age 15-24"] = acs5_12['15_to_19_years'] + acs5_12['20_to_24_years']
acs5_12["Age 25-44"] = acs5_12['25_to_34_years'] + acs5_12['35_to_44_years']
acs5_12["Age 45-64"] = acs5_12['45_to_54_years'] + acs5_12['55_to_59_years'] + acs5_12['60_to_64_years']
acs5_12["Age >= 65"] = acs5_12['65_years_and_over']


# -- drop unnecesary columns
acs5_12.drop(columns=['under_5_years', '5_to_9_years', '10_to_14_years',
                      '15_to_19_years', '20_to_24_years', '25_to_34_years',
                      '35_to_44_years', '45_to_54_years', '55_to_59_years',
                      '60_to_64_years', '65_years_and_over'], inplace=True)
acs5_12.head()

,GEOID,year,white,female,male,median_age,total_population,households,Age < 15,Age 15-24,Age 25-44,Age 45-64,Age >= 65
0,01001,2012,43841,28052,26538,37.0,54590,22077,11885,7561,14507,13945,6692
1,01003,2012,160739,93956,89270,41.2,183226,103984,34953,21002,44796,51424,31051
2,01005,2012,13816,12720,14749,38.2,27469,11878,4966,3521,7448,7518,4016
3,01007,2012,17661,10615,12154,39.4,22769,8958,4395,2856,6136,6441,2941
4,01009,2012,55359,28977,28489,39.1,57466,23761,11530,7233,14729,15418,8556


##ACS 2013

In [81]:
acs5_13 = pd.read_csv(dpath + "ACSDP5Y2013.DP05-Data.csv", skiprows=1, low_memory=False)

# -- create GEOID
acs5_13["GEOID"] = acs5_13.Geography.str[-5:]
# -- create year
acs5_13["year"] = 2013

# -- select columns
cols = ['GEOID', 'year',
         'Estimate!!RACE!!One race!!White',
         'Estimate!!SEX AND AGE!!Under 5 years',
         'Estimate!!SEX AND AGE!!5 to 9 years',
         'Estimate!!SEX AND AGE!!10 to 14 years',
         'Estimate!!SEX AND AGE!!15 to 19 years',
         'Estimate!!SEX AND AGE!!20 to 24 years',
         'Estimate!!SEX AND AGE!!25 to 34 years',
         'Estimate!!SEX AND AGE!!35 to 44 years',
         'Estimate!!SEX AND AGE!!45 to 54 years',
         'Estimate!!SEX AND AGE!!55 to 59 years',
         'Estimate!!SEX AND AGE!!60 to 64 years',
         'Estimate!!SEX AND AGE!!65 years and over',
         'Estimate!!SEX AND AGE!!Total population!!Female',
         'Estimate!!SEX AND AGE!!Total population!!Male',
         'Estimate!!SEX AND AGE!!Median age (years)',
         'Estimate!!SEX AND AGE!!Total population',
         'Estimate!!Total housing units']
acs5_13 = acs5_13[cols]

renamed_columns = {'Estimate!!RACE!!One race!!White': 'white',
         'Estimate!!SEX AND AGE!!Under 5 years': "under_5_years" ,
         'Estimate!!SEX AND AGE!!5 to 9 years': '5_to_9_years',
         'Estimate!!SEX AND AGE!!10 to 14 years': '10_to_14_years',
         'Estimate!!SEX AND AGE!!15 to 19 years': '15_to_19_years',
         'Estimate!!SEX AND AGE!!20 to 24 years': '20_to_24_years',
         'Estimate!!SEX AND AGE!!25 to 34 years': '25_to_34_years',
         'Estimate!!SEX AND AGE!!35 to 44 years': '35_to_44_years',
         'Estimate!!SEX AND AGE!!45 to 54 years': '45_to_54_years',
         'Estimate!!SEX AND AGE!!55 to 59 years': '55_to_59_years',
         'Estimate!!SEX AND AGE!!60 to 64 years': '60_to_64_years',
         'Estimate!!SEX AND AGE!!65 years and over': '65_years_and_over',
         'Estimate!!SEX AND AGE!!Total population!!Female': 'female',
         'Estimate!!SEX AND AGE!!Total population!!Male': 'male',
         'Estimate!!SEX AND AGE!!Median age (years)': 'median_age',
         'Estimate!!SEX AND AGE!!Total population': 'total_population',
         'Estimate!!Total housing units': 'households'}
acs5_13.rename(columns=renamed_columns, inplace=True)

# -- create age groups
acs5_13["Age < 15"] = acs5_13['under_5_years'] + acs5_13['5_to_9_years'] + acs5_13['10_to_14_years']
acs5_13["Age 15-24"] = acs5_13['15_to_19_years'] + acs5_13['20_to_24_years']
acs5_13["Age 25-44"] = acs5_13['25_to_34_years'] + acs5_13['35_to_44_years']
acs5_13["Age 45-64"] = acs5_13['45_to_54_years'] + acs5_13['55_to_59_years'] + acs5_13['60_to_64_years']
acs5_13["Age >= 65"] = acs5_13['65_years_and_over']


# -- drop unnecesary columns
acs5_13.drop(columns=['under_5_years', '5_to_9_years', '10_to_14_years',
                      '15_to_19_years', '20_to_24_years', '25_to_34_years',
                      '35_to_44_years', '45_to_54_years', '55_to_59_years',
                      '60_to_64_years', '65_years_and_over'], inplace=True)


##ACS 2014

In [82]:
acs5_14 = pd.read_csv(dpath + "ACSDP5Y2014.DP05-Data.csv", skiprows=1, low_memory=False)

# -- create GEOID
acs5_14["GEOID"] = acs5_14.Geography.str[-5:]
# -- create year
acs5_14["year"] = 2014

# -- select columns
cols = ['GEOID', 'year',
         'Estimate!!RACE!!One race!!White',
         'Estimate!!SEX AND AGE!!Under 5 years',
         'Estimate!!SEX AND AGE!!5 to 9 years',
         'Estimate!!SEX AND AGE!!10 to 14 years',
         'Estimate!!SEX AND AGE!!15 to 19 years',
         'Estimate!!SEX AND AGE!!20 to 24 years',
         'Estimate!!SEX AND AGE!!25 to 34 years',
         'Estimate!!SEX AND AGE!!35 to 44 years',
         'Estimate!!SEX AND AGE!!45 to 54 years',
         'Estimate!!SEX AND AGE!!55 to 59 years',
         'Estimate!!SEX AND AGE!!60 to 64 years',
         'Estimate!!SEX AND AGE!!65 years and over',
         'Estimate!!SEX AND AGE!!Total population!!Female',
         'Estimate!!SEX AND AGE!!Total population!!Male',
         'Estimate!!SEX AND AGE!!Median age (years)',
         'Estimate!!SEX AND AGE!!Total population',
         'Estimate!!Total housing units']
acs5_14 = acs5_14[cols]

renamed_columns = {'Estimate!!RACE!!One race!!White': 'white',
         'Estimate!!SEX AND AGE!!Under 5 years': "under_5_years" ,
         'Estimate!!SEX AND AGE!!5 to 9 years': '5_to_9_years',
         'Estimate!!SEX AND AGE!!10 to 14 years': '10_to_14_years',
         'Estimate!!SEX AND AGE!!15 to 19 years': '15_to_19_years',
         'Estimate!!SEX AND AGE!!20 to 24 years': '20_to_24_years',
         'Estimate!!SEX AND AGE!!25 to 34 years': '25_to_34_years',
         'Estimate!!SEX AND AGE!!35 to 44 years': '35_to_44_years',
         'Estimate!!SEX AND AGE!!45 to 54 years': '45_to_54_years',
         'Estimate!!SEX AND AGE!!55 to 59 years': '55_to_59_years',
         'Estimate!!SEX AND AGE!!60 to 64 years': '60_to_64_years',
         'Estimate!!SEX AND AGE!!65 years and over': '65_years_and_over',
         'Estimate!!SEX AND AGE!!Total population!!Female': 'female',
         'Estimate!!SEX AND AGE!!Total population!!Male': 'male',
         'Estimate!!SEX AND AGE!!Median age (years)': 'median_age',
         'Estimate!!SEX AND AGE!!Total population': 'total_population',
         'Estimate!!Total housing units': 'households'}
acs5_14.rename(columns=renamed_columns, inplace=True)

# -- create age groups
acs5_14["Age < 15"] = acs5_14['under_5_years'] + acs5_14['5_to_9_years'] + acs5_14['10_to_14_years']
acs5_14["Age 15-24"] = acs5_14['15_to_19_years'] + acs5_14['20_to_24_years']
acs5_14["Age 25-44"] = acs5_14['25_to_34_years'] + acs5_14['35_to_44_years']
acs5_14["Age 45-64"] = acs5_14['45_to_54_years'] + acs5_14['55_to_59_years'] + acs5_14['60_to_64_years']
acs5_14["Age >= 65"] = acs5_14['65_years_and_over']


# -- drop unnecesary columns
acs5_14.drop(columns=['under_5_years', '5_to_9_years', '10_to_14_years',
                      '15_to_19_years', '20_to_24_years', '25_to_34_years',
                      '35_to_44_years', '45_to_54_years', '55_to_59_years',
                      '60_to_64_years', '65_years_and_over'], inplace=True)


##ACS 2015

In [83]:
acs5_15 = pd.read_csv(dpath + "ACSDP5Y2015.DP05-Data.csv", skiprows=1, low_memory=False)

# -- create GEOID
acs5_15["GEOID"] = acs5_15.Geography.str[-5:]
# -- create year
acs5_15["year"] = 2015

# -- select columns
cols = ['GEOID', 'year',
         'Estimate!!RACE!!One race!!White',
         'Estimate!!SEX AND AGE!!Under 5 years',
         'Estimate!!SEX AND AGE!!5 to 9 years',
         'Estimate!!SEX AND AGE!!10 to 14 years',
         'Estimate!!SEX AND AGE!!15 to 19 years',
         'Estimate!!SEX AND AGE!!20 to 24 years',
         'Estimate!!SEX AND AGE!!25 to 34 years',
         'Estimate!!SEX AND AGE!!35 to 44 years',
         'Estimate!!SEX AND AGE!!45 to 54 years',
         'Estimate!!SEX AND AGE!!55 to 59 years',
         'Estimate!!SEX AND AGE!!60 to 64 years',
         'Estimate!!SEX AND AGE!!65 years and over',
         'Estimate!!SEX AND AGE!!Total population!!Female',
         'Estimate!!SEX AND AGE!!Total population!!Male',
         'Estimate!!SEX AND AGE!!Median age (years)',
         'Estimate!!SEX AND AGE!!Total population',
         'Estimate!!Total housing units']
acs5_15 = acs5_15[cols]

renamed_columns = {'Estimate!!RACE!!One race!!White': 'white',
         'Estimate!!SEX AND AGE!!Under 5 years': "under_5_years" ,
         'Estimate!!SEX AND AGE!!5 to 9 years': '5_to_9_years',
         'Estimate!!SEX AND AGE!!10 to 14 years': '10_to_14_years',
         'Estimate!!SEX AND AGE!!15 to 19 years': '15_to_19_years',
         'Estimate!!SEX AND AGE!!20 to 24 years': '20_to_24_years',
         'Estimate!!SEX AND AGE!!25 to 34 years': '25_to_34_years',
         'Estimate!!SEX AND AGE!!35 to 44 years': '35_to_44_years',
         'Estimate!!SEX AND AGE!!45 to 54 years': '45_to_54_years',
         'Estimate!!SEX AND AGE!!55 to 59 years': '55_to_59_years',
         'Estimate!!SEX AND AGE!!60 to 64 years': '60_to_64_years',
         'Estimate!!SEX AND AGE!!65 years and over': '65_years_and_over',
         'Estimate!!SEX AND AGE!!Total population!!Female': 'female',
         'Estimate!!SEX AND AGE!!Total population!!Male': 'male',
         'Estimate!!SEX AND AGE!!Median age (years)': 'median_age',
         'Estimate!!SEX AND AGE!!Total population': 'total_population',
         'Estimate!!Total housing units': 'households'}
acs5_15.rename(columns=renamed_columns, inplace=True)

# -- create age groups
acs5_15["Age < 15"] = acs5_15['under_5_years'] + acs5_15['5_to_9_years'] + acs5_15['10_to_14_years']
acs5_15["Age 15-24"] = acs5_15['15_to_19_years'] + acs5_15['20_to_24_years']
acs5_15["Age 25-44"] = acs5_15['25_to_34_years'] + acs5_15['35_to_44_years']
acs5_15["Age 45-64"] = acs5_15['45_to_54_years'] + acs5_15['55_to_59_years'] + acs5_15['60_to_64_years']
acs5_15["Age >= 65"] = acs5_15['65_years_and_over']


# -- drop unnecesary columns
acs5_15.drop(columns=['under_5_years', '5_to_9_years', '10_to_14_years',
                      '15_to_19_years', '20_to_24_years', '25_to_34_years',
                      '35_to_44_years', '45_to_54_years', '55_to_59_years',
                      '60_to_64_years', '65_years_and_over'], inplace=True)


In [84]:
acs5_15.head()

,GEOID,year,white,female,male,median_age,total_population,households,Age < 15,Age 15-24,Age 25-44,Age 45-64,Age >= 65
0,01001,2015,42741,28476,26745,37.7,55221,22582,11446,7456,14440,14401,7478
1,01003,2015,168646,99807,95314,42.2,195121,106422,36149,21971,47367,54258,35376
2,01005,2015,12756,12435,14497,38.8,26932,11810,4798,3326,7230,7253,4325
3,01007,2015,17327,10531,12073,38.9,22604,8971,4011,2964,6531,5843,3255
4,01009,2015,54881,29198,28512,40.7,57710,23860,11239,7063,14213,15683,9512


##ACS 2016

In [85]:
acs5_16 = pd.read_csv(dpath + "ACSDP5Y2016.DP05-Data.csv", skiprows=1, low_memory=False)

# -- create GEOID
acs5_16["GEOID"] = acs5_16.Geography.str[-5:]
# -- create year
acs5_16["year"] = 2016

# -- select columns
cols = ['GEOID', 'year',
         'Estimate!!RACE!!One race!!White',
         'Estimate!!SEX AND AGE!!Under 5 years',
         'Estimate!!SEX AND AGE!!5 to 9 years',
         'Estimate!!SEX AND AGE!!10 to 14 years',
         'Estimate!!SEX AND AGE!!15 to 19 years',
         'Estimate!!SEX AND AGE!!20 to 24 years',
         'Estimate!!SEX AND AGE!!25 to 34 years',
         'Estimate!!SEX AND AGE!!35 to 44 years',
         'Estimate!!SEX AND AGE!!45 to 54 years',
         'Estimate!!SEX AND AGE!!55 to 59 years',
         'Estimate!!SEX AND AGE!!60 to 64 years',
         'Estimate!!SEX AND AGE!!65 years and over',
         'Estimate!!SEX AND AGE!!Total population!!Female',
         'Estimate!!SEX AND AGE!!Total population!!Male',
         'Estimate!!SEX AND AGE!!Median age (years)',
         'Estimate!!SEX AND AGE!!Total population',
         'Estimate!!Total housing units']
acs5_16 = acs5_16[cols]

renamed_columns = {'Estimate!!RACE!!One race!!White': 'white',
         'Estimate!!SEX AND AGE!!Under 5 years': "under_5_years" ,
         'Estimate!!SEX AND AGE!!5 to 9 years': '5_to_9_years',
         'Estimate!!SEX AND AGE!!10 to 14 years': '10_to_14_years',
         'Estimate!!SEX AND AGE!!15 to 19 years': '15_to_19_years',
         'Estimate!!SEX AND AGE!!20 to 24 years': '20_to_24_years',
         'Estimate!!SEX AND AGE!!25 to 34 years': '25_to_34_years',
         'Estimate!!SEX AND AGE!!35 to 44 years': '35_to_44_years',
         'Estimate!!SEX AND AGE!!45 to 54 years': '45_to_54_years',
         'Estimate!!SEX AND AGE!!55 to 59 years': '55_to_59_years',
         'Estimate!!SEX AND AGE!!60 to 64 years': '60_to_64_years',
         'Estimate!!SEX AND AGE!!65 years and over': '65_years_and_over',
         'Estimate!!SEX AND AGE!!Total population!!Female': 'female',
         'Estimate!!SEX AND AGE!!Total population!!Male': 'male',
         'Estimate!!SEX AND AGE!!Median age (years)': 'median_age',
         'Estimate!!SEX AND AGE!!Total population': 'total_population',
         'Estimate!!Total housing units': 'households'}
acs5_16.rename(columns=renamed_columns, inplace=True)

# -- create age groups
acs5_16["Age < 15"] = acs5_16['under_5_years'] + acs5_16['5_to_9_years'] + acs5_16['10_to_14_years']
acs5_16["Age 15-24"] = acs5_16['15_to_19_years'] + acs5_16['20_to_24_years']
acs5_16["Age 25-44"] = acs5_16['25_to_34_years'] + acs5_16['35_to_44_years']
acs5_16["Age 45-64"] = acs5_16['45_to_54_years'] + acs5_16['55_to_59_years'] + acs5_16['60_to_64_years']
acs5_16["Age >= 65"] = acs5_16['65_years_and_over']


# -- drop unnecesary columns
acs5_16.drop(columns=['under_5_years', '5_to_9_years', '10_to_14_years',
                      '15_to_19_years', '20_to_24_years', '25_to_34_years',
                      '35_to_44_years', '45_to_54_years', '55_to_59_years',
                      '60_to_64_years', '65_years_and_over'], inplace=True)


##ACS 2017

In [86]:
acs5_17 = pd.read_csv(dpath + "ACSDP5Y2017.DP05-Data.csv", skiprows=1, low_memory=False)

# -- create GEOID
acs5_17["GEOID"] = acs5_17.Geography.str[-5:]
# -- create year
acs5_17["year"] = 2017

# -- select columns
cols = ['GEOID', 'year',
         'Estimate!!RACE!!Total population!!One race!!White',
         'Estimate!!SEX AND AGE!!Total population!!Under 5 years',
         'Estimate!!SEX AND AGE!!Total population!!5 to 9 years',
         'Estimate!!SEX AND AGE!!Total population!!10 to 14 years',
         'Estimate!!SEX AND AGE!!Total population!!15 to 19 years',
         'Estimate!!SEX AND AGE!!Total population!!20 to 24 years',
         'Estimate!!SEX AND AGE!!Total population!!25 to 34 years',
         'Estimate!!SEX AND AGE!!Total population!!35 to 44 years',
         'Estimate!!SEX AND AGE!!Total population!!45 to 54 years',
         'Estimate!!SEX AND AGE!!Total population!!55 to 59 years',
         'Estimate!!SEX AND AGE!!Total population!!60 to 64 years',
         'Estimate!!SEX AND AGE!!Total population!!65 years and over',
         'Estimate!!SEX AND AGE!!Total population!!Female',
         'Estimate!!SEX AND AGE!!Total population!!Male',
         'Estimate!!SEX AND AGE!!Total population!!Median age (years)',
         'Estimate!!RACE!!Total population',
         'Estimate!!Total housing units']
acs5_17 = acs5_17[cols]

renamed_columns = {'Estimate!!RACE!!Total population!!One race!!White': 'white',
         'Estimate!!SEX AND AGE!!Total population!!Under 5 years': "under_5_years" ,
         'Estimate!!SEX AND AGE!!Total population!!5 to 9 years': '5_to_9_years',
         'Estimate!!SEX AND AGE!!Total population!!10 to 14 years': '10_to_14_years',
         'Estimate!!SEX AND AGE!!Total population!!15 to 19 years': '15_to_19_years',
         'Estimate!!SEX AND AGE!!Total population!!20 to 24 years': '20_to_24_years',
         'Estimate!!SEX AND AGE!!Total population!!25 to 34 years': '25_to_34_years',
         'Estimate!!SEX AND AGE!!Total population!!35 to 44 years': '35_to_44_years',
         'Estimate!!SEX AND AGE!!Total population!!45 to 54 years': '45_to_54_years',
         'Estimate!!SEX AND AGE!!Total population!!55 to 59 years': '55_to_59_years',
         'Estimate!!SEX AND AGE!!Total population!!60 to 64 years': '60_to_64_years',
         'Estimate!!SEX AND AGE!!Total population!!65 years and over': '65_years_and_over',
         'Estimate!!SEX AND AGE!!Total population!!Female': 'female',
         'Estimate!!SEX AND AGE!!Total population!!Male': 'male',
         'Estimate!!SEX AND AGE!!Total population!!Median age (years)': 'median_age',
         'Estimate!!RACE!!Total population': 'total_population',
         'Estimate!!Total housing units': 'households'}
acs5_17 = acs5_17.rename(columns=renamed_columns).copy()

# -- create age groups
acs5_17["Age < 15"] = acs5_17['under_5_years'] + acs5_17['5_to_9_years'] + acs5_17['10_to_14_years']
acs5_17["Age 15-24"] = acs5_17['15_to_19_years'] + acs5_17['20_to_24_years']
acs5_17["Age 25-44"] = acs5_17['25_to_34_years'] + acs5_17['35_to_44_years']
acs5_17["Age 45-64"] = acs5_17['45_to_54_years'] + acs5_17['55_to_59_years'] + acs5_17['60_to_64_years']
acs5_17["Age >= 65"] = acs5_17['65_years_and_over']


# -- drop unnecesary columns
acs5_17.drop(columns=['under_5_years', '5_to_9_years', '10_to_14_years',
                      '15_to_19_years', '20_to_24_years', '25_to_34_years',
                      '35_to_44_years', '45_to_54_years', '55_to_59_years',
                      '60_to_64_years', '65_years_and_over'], inplace=True)

##ACS 2018

In [87]:
acs5_18 = pd.read_csv(dpath + "ACSDP5Y2018.DP05-Data.csv", skiprows=1, low_memory=False)

# -- create GEOID
acs5_18["GEOID"] = acs5_18.Geography.str[-5:]
# -- create year
acs5_18["year"] = 2018

# -- select columns
cols = ['GEOID', 'year',
         'Estimate!!RACE!!Total population!!One race!!White',
         'Estimate!!SEX AND AGE!!Total population!!Under 5 years',
         'Estimate!!SEX AND AGE!!Total population!!5 to 9 years',
         'Estimate!!SEX AND AGE!!Total population!!10 to 14 years',
         'Estimate!!SEX AND AGE!!Total population!!15 to 19 years',
         'Estimate!!SEX AND AGE!!Total population!!20 to 24 years',
         'Estimate!!SEX AND AGE!!Total population!!25 to 34 years',
         'Estimate!!SEX AND AGE!!Total population!!35 to 44 years',
         'Estimate!!SEX AND AGE!!Total population!!45 to 54 years',
         'Estimate!!SEX AND AGE!!Total population!!55 to 59 years',
         'Estimate!!SEX AND AGE!!Total population!!60 to 64 years',
         'Estimate!!SEX AND AGE!!Total population!!65 years and over',
         'Estimate!!SEX AND AGE!!Total population!!Female',
         'Estimate!!SEX AND AGE!!Total population!!Male',
         'Estimate!!SEX AND AGE!!Total population!!Median age (years)',
         'Estimate!!RACE!!Total population',
         'Estimate!!Total housing units']
acs5_18 = acs5_18[cols]

renamed_columns = {'Estimate!!RACE!!Total population!!One race!!White': 'white',
         'Estimate!!SEX AND AGE!!Total population!!Under 5 years': "under_5_years" ,
         'Estimate!!SEX AND AGE!!Total population!!5 to 9 years': '5_to_9_years',
         'Estimate!!SEX AND AGE!!Total population!!10 to 14 years': '10_to_14_years',
         'Estimate!!SEX AND AGE!!Total population!!15 to 19 years': '15_to_19_years',
         'Estimate!!SEX AND AGE!!Total population!!20 to 24 years': '20_to_24_years',
         'Estimate!!SEX AND AGE!!Total population!!25 to 34 years': '25_to_34_years',
         'Estimate!!SEX AND AGE!!Total population!!35 to 44 years': '35_to_44_years',
         'Estimate!!SEX AND AGE!!Total population!!45 to 54 years': '45_to_54_years',
         'Estimate!!SEX AND AGE!!Total population!!55 to 59 years': '55_to_59_years',
         'Estimate!!SEX AND AGE!!Total population!!60 to 64 years': '60_to_64_years',
         'Estimate!!SEX AND AGE!!Total population!!65 years and over': '65_years_and_over',
         'Estimate!!SEX AND AGE!!Total population!!Female': 'female',
         'Estimate!!SEX AND AGE!!Total population!!Male': 'male',
         'Estimate!!SEX AND AGE!!Total population!!Median age (years)': 'median_age',
         'Estimate!!RACE!!Total population': 'total_population',
         'Estimate!!Total housing units': 'households'}
acs5_18.rename(columns=renamed_columns, inplace=True)

# -- create age groups
acs5_18["Age < 15"] = acs5_18['under_5_years'] + acs5_18['5_to_9_years'] + acs5_18['10_to_14_years']
acs5_18["Age 15-24"] = acs5_18['15_to_19_years'] + acs5_18['20_to_24_years']
acs5_18["Age 25-44"] = acs5_18['25_to_34_years'] + acs5_18['35_to_44_years']
acs5_18["Age 45-64"] = acs5_18['45_to_54_years'] + acs5_18['55_to_59_years'] + acs5_18['60_to_64_years']
acs5_18["Age >= 65"] = acs5_18['65_years_and_over']


# -- drop unnecesary columns
acs5_18.drop(columns=['under_5_years', '5_to_9_years', '10_to_14_years',
                      '15_to_19_years', '20_to_24_years', '25_to_34_years',
                      '35_to_44_years', '45_to_54_years', '55_to_59_years',
                      '60_to_64_years', '65_years_and_over'], inplace=True)

In [88]:
acs5_18.head()

,GEOID,year,white,female,male,median_age,total_population,households,Age < 15,Age 15-24,Age 25-44,Age 45-64,Age >= 65
0,01001,2018,42437,28326,26874,37.8,55200,23315,10842,7192,14438,14678,8050
1,01003,2018,179526,106919,101188,42.8,208107,111945,37621,23497,48703,57621,40665
2,01005,2018,12216,12085,13697,39.9,25782,11937,4517,3092,6779,6760,4634
3,01007,2018,17268,10375,12152,39.9,22527,9161,3742,3005,5970,6149,3661
4,01009,2018,55054,29211,28434,40.8,57645,24222,11112,6906,13939,15455,10233


##ACS 2019

In [89]:
acs5_19 = pd.read_csv(dpath + "ACSDP5Y2019.DP05-Data.csv", skiprows=1, low_memory=False)

# -- create GEOID
acs5_19["GEOID"] = acs5_19.Geography.str[-5:]
# -- create year
acs5_19["year"] = 2019

# -- select columns
cols = ['GEOID', 'year',
         'Estimate!!RACE!!Total population!!One race!!White',
         'Estimate!!SEX AND AGE!!Total population!!Under 5 years',
         'Estimate!!SEX AND AGE!!Total population!!5 to 9 years',
         'Estimate!!SEX AND AGE!!Total population!!10 to 14 years',
         'Estimate!!SEX AND AGE!!Total population!!15 to 19 years',
         'Estimate!!SEX AND AGE!!Total population!!20 to 24 years',
         'Estimate!!SEX AND AGE!!Total population!!25 to 34 years',
         'Estimate!!SEX AND AGE!!Total population!!35 to 44 years',
         'Estimate!!SEX AND AGE!!Total population!!45 to 54 years',
         'Estimate!!SEX AND AGE!!Total population!!55 to 59 years',
         'Estimate!!SEX AND AGE!!Total population!!60 to 64 years',
         'Estimate!!SEX AND AGE!!Total population!!65 years and over',
         'Estimate!!SEX AND AGE!!Total population!!Female',
         'Estimate!!SEX AND AGE!!Total population!!Male',
         'Estimate!!SEX AND AGE!!Total population!!Median age (years)',
         'Estimate!!RACE!!Total population',
         'Estimate!!Total housing units']
acs5_19 = acs5_19[cols]

renamed_columns = {'Estimate!!RACE!!Total population!!One race!!White': 'white',
         'Estimate!!SEX AND AGE!!Total population!!Under 5 years': "under_5_years" ,
         'Estimate!!SEX AND AGE!!Total population!!5 to 9 years': '5_to_9_years',
         'Estimate!!SEX AND AGE!!Total population!!10 to 14 years': '10_to_14_years',
         'Estimate!!SEX AND AGE!!Total population!!15 to 19 years': '15_to_19_years',
         'Estimate!!SEX AND AGE!!Total population!!20 to 24 years': '20_to_24_years',
         'Estimate!!SEX AND AGE!!Total population!!25 to 34 years': '25_to_34_years',
         'Estimate!!SEX AND AGE!!Total population!!35 to 44 years': '35_to_44_years',
         'Estimate!!SEX AND AGE!!Total population!!45 to 54 years': '45_to_54_years',
         'Estimate!!SEX AND AGE!!Total population!!55 to 59 years': '55_to_59_years',
         'Estimate!!SEX AND AGE!!Total population!!60 to 64 years': '60_to_64_years',
         'Estimate!!SEX AND AGE!!Total population!!65 years and over': '65_years_and_over',
         'Estimate!!SEX AND AGE!!Total population!!Female': 'female',
         'Estimate!!SEX AND AGE!!Total population!!Male': 'male',
         'Estimate!!SEX AND AGE!!Total population!!Median age (years)': 'median_age',
         'Estimate!!RACE!!Total population': 'total_population',
         'Estimate!!Total housing units': 'households'}
acs5_19.rename(columns=renamed_columns, inplace=True)

# -- create age groups
acs5_19["Age < 15"]  = acs5_19['under_5_years']  + acs5_19['5_to_9_years']  + acs5_19['10_to_14_years']
acs5_19["Age 15-24"] = acs5_19['15_to_19_years'] + acs5_19['20_to_24_years']
acs5_19["Age 25-44"] = acs5_19['25_to_34_years'] + acs5_19['35_to_44_years']
acs5_19["Age 45-64"] = acs5_19['45_to_54_years'] + acs5_19['55_to_59_years'] + acs5_19['60_to_64_years']
acs5_19["Age >= 65"] = acs5_19['65_years_and_over']


# -- drop unnecesary columns
acs5_19.drop(columns=['under_5_years', '5_to_9_years', '10_to_14_years',
                      '15_to_19_years', '20_to_24_years', '25_to_34_years',
                      '35_to_44_years', '45_to_54_years', '55_to_59_years',
                      '60_to_64_years', '65_years_and_over'], inplace=True)

In [90]:
acs5_19.head()

,GEOID,year,white,female,male,median_age,total_population,households,Age < 15,Age 15-24,Age 25-44,Age 45-64,Age >= 65
0,01001,2019,42527,28446,26934,38.2,55380,23493,10631,7382,14362,14722,8283
1,01003,2019,183471,109334,103496,43.0,212830,114164,38009,23709,49776,58805,42531
2,01005,2019,11869,11940,13421,40.4,25361,12013,4393,3004,6680,6574,4710
3,01007,2019,17272,10343,12150,40.9,22493,9185,3666,2659,6186,6398,3584
4,01009,2019,55062,29186,28495,40.7,57681,24323,11056,6834,14102,15363,10326


##ACS 2020

In [91]:
acs5_20 = pd.read_csv(dpath + "ACSDP5Y2020.DP05-Data.csv", skiprows=1, low_memory=False)

# -- create GEOID
acs5_20["GEOID"] = acs5_20.Geography.str[-5:]
# -- create year
acs5_20["year"] = 2020

# -- select columns
cols = ['GEOID', 'year',
         'Estimate!!RACE!!Total population!!One race!!White',
         'Estimate!!SEX AND AGE!!Total population!!Under 5 years',
         'Estimate!!SEX AND AGE!!Total population!!5 to 9 years',
         'Estimate!!SEX AND AGE!!Total population!!10 to 14 years',
         'Estimate!!SEX AND AGE!!Total population!!15 to 19 years',
         'Estimate!!SEX AND AGE!!Total population!!20 to 24 years',
         'Estimate!!SEX AND AGE!!Total population!!25 to 34 years',
         'Estimate!!SEX AND AGE!!Total population!!35 to 44 years',
         'Estimate!!SEX AND AGE!!Total population!!45 to 54 years',
         'Estimate!!SEX AND AGE!!Total population!!55 to 59 years',
         'Estimate!!SEX AND AGE!!Total population!!60 to 64 years',
         'Estimate!!SEX AND AGE!!Total population!!65 years and over',
         'Estimate!!SEX AND AGE!!Total population!!Female',
         'Estimate!!SEX AND AGE!!Total population!!Male',
         'Estimate!!SEX AND AGE!!Total population!!Median age (years)',
         'Estimate!!RACE!!Total population',
         'Estimate!!Total housing units']
acs5_20 = acs5_20[cols]

renamed_columns = {'Estimate!!RACE!!Total population!!One race!!White': 'white',
         'Estimate!!SEX AND AGE!!Total population!!Under 5 years': "under_5_years" ,
         'Estimate!!SEX AND AGE!!Total population!!5 to 9 years': '5_to_9_years',
         'Estimate!!SEX AND AGE!!Total population!!10 to 14 years': '10_to_14_years',
         'Estimate!!SEX AND AGE!!Total population!!15 to 19 years': '15_to_19_years',
         'Estimate!!SEX AND AGE!!Total population!!20 to 24 years': '20_to_24_years',
         'Estimate!!SEX AND AGE!!Total population!!25 to 34 years': '25_to_34_years',
         'Estimate!!SEX AND AGE!!Total population!!35 to 44 years': '35_to_44_years',
         'Estimate!!SEX AND AGE!!Total population!!45 to 54 years': '45_to_54_years',
         'Estimate!!SEX AND AGE!!Total population!!55 to 59 years': '55_to_59_years',
         'Estimate!!SEX AND AGE!!Total population!!60 to 64 years': '60_to_64_years',
         'Estimate!!SEX AND AGE!!Total population!!65 years and over': '65_years_and_over',
         'Estimate!!SEX AND AGE!!Total population!!Female': 'female',
         'Estimate!!SEX AND AGE!!Total population!!Male': 'male',
         'Estimate!!SEX AND AGE!!Total population!!Median age (years)': 'median_age',
         'Estimate!!RACE!!Total population': 'total_population',
         'Estimate!!Total housing units': 'households'}
acs5_20.rename(columns=renamed_columns, inplace=True)

# -- create age groups
acs5_20["Age < 15"]  = acs5_20['under_5_years']  + acs5_20['5_to_9_years']  + acs5_20['10_to_14_years']
acs5_20["Age 15-24"] = acs5_20['15_to_19_years'] + acs5_20['20_to_24_years']
acs5_20["Age 25-44"] = acs5_20['25_to_34_years'] + acs5_20['35_to_44_years']
acs5_20["Age 45-64"] = acs5_20['45_to_54_years'] + acs5_20['55_to_59_years'] + acs5_20['60_to_64_years']
acs5_20["Age >= 65"] = acs5_20['65_years_and_over']


# -- drop unnecesary columns
acs5_20.drop(columns=['under_5_years', '5_to_9_years', '10_to_14_years',
                      '15_to_19_years', '20_to_24_years', '25_to_34_years',
                      '35_to_44_years', '45_to_54_years', '55_to_59_years',
                      '60_to_64_years', '65_years_and_over'], inplace=True)

##ACS 2021

In [92]:
acs5_21 = pd.read_csv(dpath + "ACSDP5Y2021.DP05-Data.csv", skiprows=1, low_memory=False)

# -- create GEOID
acs5_21["GEOID"] = acs5_21.Geography.str[-5:]
# -- create year
acs5_21["year"] = 2021

# -- select columns
cols = ['GEOID', 'year',
         'Estimate!!RACE!!Total population!!One race!!White',
         'Estimate!!SEX AND AGE!!Total population!!Under 5 years',
         'Estimate!!SEX AND AGE!!Total population!!5 to 9 years',
         'Estimate!!SEX AND AGE!!Total population!!10 to 14 years',
         'Estimate!!SEX AND AGE!!Total population!!15 to 19 years',
         'Estimate!!SEX AND AGE!!Total population!!20 to 24 years',
         'Estimate!!SEX AND AGE!!Total population!!25 to 34 years',
         'Estimate!!SEX AND AGE!!Total population!!35 to 44 years',
         'Estimate!!SEX AND AGE!!Total population!!45 to 54 years',
         'Estimate!!SEX AND AGE!!Total population!!55 to 59 years',
         'Estimate!!SEX AND AGE!!Total population!!60 to 64 years',
         'Estimate!!SEX AND AGE!!Total population!!65 years and over',
         'Estimate!!SEX AND AGE!!Total population!!Female',
         'Estimate!!SEX AND AGE!!Total population!!Male',
         'Estimate!!SEX AND AGE!!Total population!!Median age (years)',
         'Estimate!!RACE!!Total population',
         'Estimate!!Total housing units']
acs5_21 = acs5_21[cols]

renamed_columns = {'Estimate!!RACE!!Total population!!One race!!White': 'white',
         'Estimate!!SEX AND AGE!!Total population!!Under 5 years': "under_5_years" ,
         'Estimate!!SEX AND AGE!!Total population!!5 to 9 years': '5_to_9_years',
         'Estimate!!SEX AND AGE!!Total population!!10 to 14 years': '10_to_14_years',
         'Estimate!!SEX AND AGE!!Total population!!15 to 19 years': '15_to_19_years',
         'Estimate!!SEX AND AGE!!Total population!!20 to 24 years': '20_to_24_years',
         'Estimate!!SEX AND AGE!!Total population!!25 to 34 years': '25_to_34_years',
         'Estimate!!SEX AND AGE!!Total population!!35 to 44 years': '35_to_44_years',
         'Estimate!!SEX AND AGE!!Total population!!45 to 54 years': '45_to_54_years',
         'Estimate!!SEX AND AGE!!Total population!!55 to 59 years': '55_to_59_years',
         'Estimate!!SEX AND AGE!!Total population!!60 to 64 years': '60_to_64_years',
         'Estimate!!SEX AND AGE!!Total population!!65 years and over': '65_years_and_over',
         'Estimate!!SEX AND AGE!!Total population!!Female': 'female',
         'Estimate!!SEX AND AGE!!Total population!!Male': 'male',
         'Estimate!!SEX AND AGE!!Total population!!Median age (years)': 'median_age',
         'Estimate!!RACE!!Total population': 'total_population',
         'Estimate!!Total housing units': 'households'}
acs5_21.rename(columns=renamed_columns, inplace=True)

# -- create age groups
acs5_21["Age < 15"]  = acs5_21['under_5_years']  + acs5_21['5_to_9_years']  + acs5_21['10_to_14_years']
acs5_21["Age 15-24"] = acs5_21['15_to_19_years'] + acs5_21['20_to_24_years']
acs5_21["Age 25-44"] = acs5_21['25_to_34_years'] + acs5_21['35_to_44_years']
acs5_21["Age 45-64"] = acs5_21['45_to_54_years'] + acs5_21['55_to_59_years'] + acs5_21['60_to_64_years']
acs5_21["Age >= 65"] = acs5_21['65_years_and_over']


# -- drop unnecesary columns
acs5_21.drop(columns=['under_5_years', '5_to_9_years', '10_to_14_years',
                      '15_to_19_years', '20_to_24_years', '25_to_34_years',
                      '35_to_44_years', '45_to_54_years', '55_to_59_years',
                      '60_to_64_years', '65_years_and_over'], inplace=True)

In [93]:
acs5_21.head()

,GEOID,year,white,female,male,median_age,total_population,households,Age < 15,Age 15-24,Age 25-44,Age 45-64,Age >= 65
0,01001,2021,43755,30033,28206,38.5,58239,24170,11038,7587,15178,15621,8815
1,01003,2021,192034,116350,110781,43.4,227131,121763,40172,24982,52642,62530,46805
2,01005,2021,11495,11898,13361,40.2,25259,11667,4343,2921,6744,6450,4801
3,01007,2021,17020,10112,12300,39.7,22412,9013,3700,2655,6396,6067,3594
4,01009,2021,54439,29354,29530,41.1,58884,24527,11239,6977,14169,15915,10584


##ACS 2022

In [94]:
acs5_22 = pd.read_csv(dpath + "ACSDP5Y2022.DP05-Data.csv", skiprows=1, low_memory=False)

# -- create GEOID
acs5_22["GEOID"] = acs5_22.Geography.str[-5:]
# -- create year
acs5_22["year"] = 2022

# -- select columns
cols = ['GEOID', 'year',
         'Estimate!!RACE!!Total population!!One race!!White',
         'Estimate!!SEX AND AGE!!Total population!!Under 5 years',
         'Estimate!!SEX AND AGE!!Total population!!5 to 9 years',
         'Estimate!!SEX AND AGE!!Total population!!10 to 14 years',
         'Estimate!!SEX AND AGE!!Total population!!15 to 19 years',
         'Estimate!!SEX AND AGE!!Total population!!20 to 24 years',
         'Estimate!!SEX AND AGE!!Total population!!25 to 34 years',
         'Estimate!!SEX AND AGE!!Total population!!35 to 44 years',
         'Estimate!!SEX AND AGE!!Total population!!45 to 54 years',
         'Estimate!!SEX AND AGE!!Total population!!55 to 59 years',
         'Estimate!!SEX AND AGE!!Total population!!60 to 64 years',
         'Estimate!!SEX AND AGE!!Total population!!65 years and over',
         'Estimate!!SEX AND AGE!!Total population!!Female',
         'Estimate!!SEX AND AGE!!Total population!!Male',
         'Estimate!!SEX AND AGE!!Total population!!Median age (years)',
         'Estimate!!RACE!!Total population',
         'Estimate!!Total housing units']
acs5_22 = acs5_22[cols]

renamed_columns = {'Estimate!!RACE!!Total population!!One race!!White': 'white',
         'Estimate!!SEX AND AGE!!Total population!!Under 5 years': "under_5_years" ,
         'Estimate!!SEX AND AGE!!Total population!!5 to 9 years': '5_to_9_years',
         'Estimate!!SEX AND AGE!!Total population!!10 to 14 years': '10_to_14_years',
         'Estimate!!SEX AND AGE!!Total population!!15 to 19 years': '15_to_19_years',
         'Estimate!!SEX AND AGE!!Total population!!20 to 24 years': '20_to_24_years',
         'Estimate!!SEX AND AGE!!Total population!!25 to 34 years': '25_to_34_years',
         'Estimate!!SEX AND AGE!!Total population!!35 to 44 years': '35_to_44_years',
         'Estimate!!SEX AND AGE!!Total population!!45 to 54 years': '45_to_54_years',
         'Estimate!!SEX AND AGE!!Total population!!55 to 59 years': '55_to_59_years',
         'Estimate!!SEX AND AGE!!Total population!!60 to 64 years': '60_to_64_years',
         'Estimate!!SEX AND AGE!!Total population!!65 years and over': '65_years_and_over',
         'Estimate!!SEX AND AGE!!Total population!!Female': 'female',
         'Estimate!!SEX AND AGE!!Total population!!Male': 'male',
         'Estimate!!SEX AND AGE!!Total population!!Median age (years)': 'median_age',
         'Estimate!!RACE!!Total population': 'total_population',
         'Estimate!!Total housing units': 'households'}
acs5_22.rename(columns=renamed_columns, inplace=True)

# -- create age groups
acs5_22["Age < 15"]  = acs5_22['under_5_years']  + acs5_22['5_to_9_years']  + acs5_22['10_to_14_years']
acs5_22["Age 15-24"] = acs5_22['15_to_19_years'] + acs5_22['20_to_24_years']
acs5_22["Age 25-44"] = acs5_22['25_to_34_years'] + acs5_22['35_to_44_years']
acs5_22["Age 45-64"] = acs5_22['45_to_54_years'] + acs5_22['55_to_59_years'] + acs5_22['60_to_64_years']
acs5_22["Age >= 65"] = acs5_22['65_years_and_over']


# -- drop unnecesary columns
acs5_22.drop(columns=['under_5_years', '5_to_9_years', '10_to_14_years',
                      '15_to_19_years', '20_to_24_years', '25_to_34_years',
                      '35_to_44_years', '45_to_54_years', '55_to_59_years',
                      '60_to_64_years', '65_years_and_over'], inplace=True)

#Data Preparation

In [95]:
# It sees that there was a mistake in the data collection phase that I could not resolve before.
# The issue was that the dataset acs5_10 has the age categories in percentages.
# This portion of the code will solve that.

temp = acs5_10.copy()
temp['Age < 15'] = temp['Age < 15'] * 0.01 * temp['total_population']
temp['Age 15-24'] = temp['Age 15-24'] * 0.01 * temp['total_population']
temp['Age 25-44'] = temp['Age 25-44'] * 0.01 * temp['total_population']
temp['Age 45-64'] = temp['Age 45-64'] * 0.01 * temp['total_population']
temp['Age >= 65'] = temp['Age >= 65'] * 0.01 * temp['total_population']

acs5_10 = temp.copy()
del temp

In [96]:
# 2010 Add variable for travel time to work
dpath = '/content/drive/MyDrive/Disertation/Census/Commuting/'
fname = 'ACSST5Y2010.S0801-Data.csv'
temp = pd.read_csv(dpath+fname, skiprows=1, low_memory=False)
temp['GEOID'] = temp['Geography'].str[-5:]
cols = ['GEOID', 'Total!!Estimate!!Workers 16 years and over',
'Total!!Estimate!!Workers 16 years and over who did not work at home',
'Total!!Estimate!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)']
temp = temp[cols].copy()
temp.rename(columns={'Total!!Estimate!!Workers 16 years and over': 'workforce',
'Total!!Estimate!!Workers 16 years and over who did not work at home': 'not_remote',
'Total!!Estimate!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)': 'commuting_time'},
            inplace=True)
acs5_10 = acs5_10.merge(temp, on="GEOID", how='left').copy()
del temp

#2011 Add variable for travel time to work
dpath = '/content/drive/MyDrive/Disertation/Census/Commuting/'
fname = 'ACSST5Y2011.S0801-Data.csv'
temp = pd.read_csv(dpath+fname, skiprows=1, low_memory=False)
temp['GEOID'] = temp['Geography'].str[-5:]
cols = ['GEOID', 'Total!!Estimate!!Workers 16 years and over',
'Total!!Estimate!!Workers 16 years and over who did not work at home',
'Total!!Estimate!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)']
temp = temp[cols].copy()
temp.rename(columns={'Total!!Estimate!!Workers 16 years and over': 'workforce',
'Total!!Estimate!!Workers 16 years and over who did not work at home': 'not_remote',
'Total!!Estimate!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)': 'commuting_time'},
            inplace=True)
acs5_11 = acs5_11.merge(temp, on="GEOID", how='left').copy()
del temp

#2012 Add variable for travel time to work
dpath = '/content/drive/MyDrive/Disertation/Census/Commuting/'
fname = 'ACSST5Y2012.S0801-Data.csv'
temp = pd.read_csv(dpath+fname, skiprows=1, low_memory=False)
temp['GEOID'] = temp['Geography'].str[-5:]
cols = ['GEOID', 'Total!!Estimate!!Workers 16 years and over',
'Total!!Estimate!!Workers 16 years and over who did not work at home',
'Total!!Estimate!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)']
temp = temp[cols].copy()
temp.rename(columns={'Total!!Estimate!!Workers 16 years and over': 'workforce',
'Total!!Estimate!!Workers 16 years and over who did not work at home': 'not_remote',
'Total!!Estimate!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)': 'commuting_time'},
            inplace=True)
acs5_12 = acs5_12.merge(temp, on="GEOID", how='left').copy()
del temp

#2013 Add variable for travel time to work
dpath = '/content/drive/MyDrive/Disertation/Census/Commuting/'
fname = 'ACSST5Y2013.S0801-Data.csv'
temp = pd.read_csv(dpath+fname, skiprows=1, low_memory=False)
temp['GEOID'] = temp['Geography'].str[-5:]
cols = ['GEOID', 'Total!!Estimate!!Workers 16 years and over',
'Total!!Estimate!!Workers 16 years and over who did not work at home',
'Total!!Estimate!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)']
temp = temp[cols].copy()
temp.rename(columns={'Total!!Estimate!!Workers 16 years and over': 'workforce',
'Total!!Estimate!!Workers 16 years and over who did not work at home': 'not_remote',
'Total!!Estimate!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)': 'commuting_time'},
            inplace=True)
acs5_13 = acs5_13.merge(temp, on="GEOID", how='left').copy()
del temp

#2014 Add variable for travel time to work
dpath = '/content/drive/MyDrive/Disertation/Census/Commuting/'
fname = 'ACSST5Y2014.S0801-Data.csv'
temp = pd.read_csv(dpath+fname, skiprows=1, low_memory=False)
temp['GEOID'] = temp['Geography'].str[-5:]
cols = ['GEOID', 'Total!!Estimate!!Workers 16 years and over',
'Total!!Estimate!!Workers 16 years and over who did not work at home',
'Total!!Estimate!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)']
temp = temp[cols].copy()
temp.rename(columns={'Total!!Estimate!!Workers 16 years and over': 'workforce',
'Total!!Estimate!!Workers 16 years and over who did not work at home': 'not_remote',
'Total!!Estimate!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)': 'commuting_time'},
            inplace=True)
acs5_14 = acs5_14.merge(temp, on="GEOID", how='left').copy()
del temp

#2015 Add variable for travel time to work
dpath = '/content/drive/MyDrive/Disertation/Census/Commuting/'
fname = 'ACSST5Y2015.S0801-Data.csv'
temp = pd.read_csv(dpath+fname, skiprows=1, low_memory=False)
temp['GEOID'] = temp['Geography'].str[-5:]
cols = ['GEOID', 'Total!!Estimate!!Workers 16 years and over',
'Total!!Estimate!!Workers 16 years and over who did not work at home',
'Total!!Estimate!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)']
temp = temp[cols].copy()
temp.rename(columns={'Total!!Estimate!!Workers 16 years and over': 'workforce',
'Total!!Estimate!!Workers 16 years and over who did not work at home': 'not_remote',
'Total!!Estimate!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)': 'commuting_time'},
            inplace=True)
acs5_15 = acs5_15.merge(temp, on="GEOID", how='left').copy()
del temp

#2016 Add variable for travel time to work
dpath = '/content/drive/MyDrive/Disertation/Census/Commuting/'
fname = 'ACSST5Y2016.S0801-Data.csv'
temp = pd.read_csv(dpath+fname, skiprows=1, low_memory=False)
temp['GEOID'] = temp['Geography'].str[-5:]
cols = ['GEOID', 'Total!!Estimate!!Workers 16 years and over',
'Total!!Estimate!!Workers 16 years and over who did not work at home',
'Total!!Estimate!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)']
temp = temp[cols].copy()
temp.rename(columns={'Total!!Estimate!!Workers 16 years and over': 'workforce',
'Total!!Estimate!!Workers 16 years and over who did not work at home': 'not_remote',
'Total!!Estimate!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)': 'commuting_time'},
            inplace=True)
acs5_16 = acs5_16.merge(temp, on="GEOID", how='left').copy()
del temp

#2017 Add variable for travel time to work
dpath = '/content/drive/MyDrive/Disertation/Census/Commuting/'
fname = 'ACSST5Y2017.S0801-Data.csv'
temp = pd.read_csv(dpath+fname, skiprows=1, low_memory=False)
temp['GEOID'] = temp['Geography'].str[-5:]
cols = ['GEOID', 'Estimate!!Total!!Workers 16 years and over',
        'Estimate!!Total!!Workers 16 years and over who did not work at home',
        'Estimate!!Total!!Workers 16 years and over who did not work at home!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)']
temp = temp[cols].copy()
temp.rename(columns={'Estimate!!Total!!Workers 16 years and over': 'workforce',
  'Estimate!!Total!!Workers 16 years and over who did not work at home': 'not_remote',
  'Estimate!!Total!!Workers 16 years and over who did not work at home!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)': 'commuting_time'},
            inplace=True)
acs5_17 = acs5_17.merge(temp, on="GEOID", how='left').copy()
del temp

#2018 Add variable for travel time to work
dpath = '/content/drive/MyDrive/Disertation/Census/Commuting/'
fname = 'ACSST5Y2018.S0801-Data.csv'
temp = pd.read_csv(dpath+fname, skiprows=1, low_memory=False)
temp['GEOID'] = temp['Geography'].str[-5:]
cols = ['GEOID', 'Estimate!!Total!!Workers 16 years and over',
        'Estimate!!Total!!Workers 16 years and over who did not work at home',
        'Estimate!!Total!!Workers 16 years and over who did not work at home!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)']
temp = temp[cols].copy()
temp.rename(columns={'Estimate!!Total!!Workers 16 years and over': 'workforce',
  'Estimate!!Total!!Workers 16 years and over who did not work at home': 'not_remote',
  'Estimate!!Total!!Workers 16 years and over who did not work at home!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)': 'commuting_time'},
            inplace=True)
acs5_18 = acs5_18.merge(temp, on="GEOID", how='left').copy()
del temp

#2019 Add variable for travel time to work
dpath = '/content/drive/MyDrive/Disertation/Census/Commuting/'
fname = 'ACSST5Y2019.S0801-Data.csv'
temp = pd.read_csv(dpath+fname, skiprows=1, low_memory=False)
temp['GEOID'] = temp['Geography'].str[-5:]
cols = ['GEOID', 'Estimate!!Total!!Workers 16 years and over',
        'Estimate!!Total!!Workers 16 years and over who did not work from home',
        'Estimate!!Total!!Workers 16 years and over who did not work from home!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)']
temp = temp[cols].copy()
temp.rename(columns={'Estimate!!Total!!Workers 16 years and over': 'workforce',
  'Estimate!!Total!!Workers 16 years and over who did not work from home': 'not_remote',
  'Estimate!!Total!!Workers 16 years and over who did not work from home!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)': 'commuting_time'},
            inplace=True)
acs5_19 = acs5_19.merge(temp, on="GEOID", how='left').copy()
del temp

#2020 Add variable for travel time to work
dpath = '/content/drive/MyDrive/Disertation/Census/Commuting/'
fname = 'ACSST5Y2020.S0801-Data.csv'
temp = pd.read_csv(dpath+fname, skiprows=1, low_memory=False)
temp['GEOID'] = temp['Geography'].str[-5:]
cols = ['GEOID', 'Estimate!!Total!!Workers 16 years and over',
        'Estimate!!Total!!Workers 16 years and over who did not work from home',
        'Estimate!!Total!!Workers 16 years and over who did not work from home!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)']
temp = temp[cols].copy()
temp.rename(columns={'Estimate!!Total!!Workers 16 years and over': 'workforce',
  'Estimate!!Total!!Workers 16 years and over who did not work from home': 'not_remote',
  'Estimate!!Total!!Workers 16 years and over who did not work from home!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)': 'commuting_time'},
            inplace=True)
acs5_20 = acs5_20.merge(temp, on="GEOID", how='left').copy()
del temp

#2021 Add variable for travel time to work
dpath = '/content/drive/MyDrive/Disertation/Census/Commuting/'
fname = 'ACSST5Y2021.S0801-Data.csv'
temp = pd.read_csv(dpath+fname, skiprows=1, low_memory=False)
temp['GEOID'] = temp['Geography'].str[-5:]
cols = ['GEOID', 'Estimate!!Total!!Workers 16 years and over',
        'Estimate!!Total!!Workers 16 years and over who did not work from home',
        'Estimate!!Total!!Workers 16 years and over who did not work from home!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)']
temp = temp[cols].copy()
temp.rename(columns={'Estimate!!Total!!Workers 16 years and over': 'workforce',
  'Estimate!!Total!!Workers 16 years and over who did not work from home': 'not_remote',
  'Estimate!!Total!!Workers 16 years and over who did not work from home!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)': 'commuting_time'},
            inplace=True)
acs5_21 = acs5_21.merge(temp, on="GEOID", how='left').copy()
del temp

#2022 Add variable for travel time to work
dpath = '/content/drive/MyDrive/Disertation/Census/Commuting/'
fname = 'ACSST5Y2022.S0801-Data.csv'
temp = pd.read_csv(dpath+fname, skiprows=1, low_memory=False)
temp['GEOID'] = temp['Geography'].str[-5:]
cols = ['GEOID', 'Estimate!!Total!!Workers 16 years and over',
        'Estimate!!Total!!Workers 16 years and over who did not work from home',
        'Estimate!!Total!!Workers 16 years and over who did not work from home!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)']
temp = temp[cols].copy()
temp.rename(columns={'Estimate!!Total!!Workers 16 years and over': 'workforce',
  'Estimate!!Total!!Workers 16 years and over who did not work from home': 'not_remote',
  'Estimate!!Total!!Workers 16 years and over who did not work from home!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)': 'commuting_time'},
            inplace=True)
acs5_22 = acs5_22.merge(temp, on="GEOID", how='left').copy()
del temp

In [97]:
# 2000 Decennial Census
# Add variable for travel time to work
dpath = '/content/drive/MyDrive/Disertation/Census/Commuting/'
fname = 'DECENNIALDPSF42000.DP3-Data.csv'
temp = pd.read_csv(dpath+fname, skiprows=1, low_memory=False)
temp['GEOID'] = temp['Geography'].str[-5:]
temp = temp[temp['Race/Ethnic Group'] == 1].copy()
cols = ['GEOID', 'Number!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force',
        'Number!!COMMUTING TO WORK!!Workers 16 years and over!!Worked at home',
        'Number!!COMMUTING TO WORK!!Workers 16 years and over!!Mean travel time to work (minutes)']
temp = temp[cols].copy()
temp.rename(columns={'Number!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force': 'workforce',
  'Number!!COMMUTING TO WORK!!Workers 16 years and over!!Worked at home': 'remote',
  'Number!!COMMUTING TO WORK!!Workers 16 years and over!!Mean travel time to work (minutes)': 'commuting_time'},
            inplace=True)
temp["not_remote"] = temp['workforce'] - temp['remote']
temp = temp.drop(columns='remote')

decennial_00 = decennial_00.merge(temp, on="GEOID", how='left').copy()
del temp

In [98]:
acs5_11.head()

,GEOID,year,white,female,male,median_age,total_population,households,Age < 15,Age 15-24,Age 25-44,Age 45-64,Age >= 65,workforce,not_remote,commuting_time
0,01001,2011,43366,27770,26174,36.4,53944,21859,11872,7491,14690,13604,6287,24349,23748,25.3
1,01003,2011,157048,91970,87553,41.4,179523,102978,34468,20605,44138,50302,30010,78770,76182,25.5
2,01005,2011,13956,12897,14649,38.3,27546,11903,5046,3576,7511,7487,3926,9447,9221,23.4
3,01007,2011,17458,10898,11848,39.1,22746,8933,4435,3020,6098,6323,2870,8930,8761,27.8
4,01009,2011,55182,28783,28357,38.8,57140,23728,11573,7094,14942,15280,8251,23566,22881,33.5


In [99]:
cols = ['GEOID', 'year', 'total_population', 'white', 'male', 'female',
       'median_age', 'households', 'Age < 15', 'Age 15-24', 'Age 25-44',
       'Age 45-64', 'Age >= 65', 'workforce', 'commuting_time', 'not_remote']
decennial_00 = decennial_00[cols].copy()
acs5_10 = acs5_10[cols].copy()
acs5_11 = acs5_11[cols].copy()
acs5_12 = acs5_12[cols].copy()
acs5_13 = acs5_13[cols].copy()
acs5_14 = acs5_14[cols].copy()
acs5_15 = acs5_15[cols].copy()
acs5_16 = acs5_16[cols].copy()
acs5_17 = acs5_17[cols].copy()
acs5_18 = acs5_18[cols].copy()
acs5_19 = acs5_19[cols].copy()
acs5_20 = acs5_20[cols].copy()
acs5_21 = acs5_21[cols].copy()
acs5_22 = acs5_22[cols].copy()




In [100]:
dataframes = [decennial_00, acs5_10, acs5_11, acs5_12, acs5_13, acs5_14, acs5_15,
              acs5_16, acs5_17, acs5_18, acs5_19, acs5_20, acs5_21, acs5_22].copy()

del decennial_00, acs5_10, acs5_11, acs5_12, acs5_13, acs5_14, acs5_15,
del acs5_16, acs5_17, acs5_18, acs5_19, acs5_20, acs5_21, acs5_22

control = pd.concat(dataframes)

del dataframes


,GEOID,year,total_population,white,male,female,median_age,households,Age < 15,Age 15-24,Age 25-44,Age 45-64,Age >= 65,workforce,commuting_time,not_remote
0,01001,2000,43671,34960,21177,22494,35.1,15972,10468.0,5614.0,13304.0,9843.0,4442.0,21167.0,26.5,20795.0
1,01003,2000,140415,122349,68682,71733,39.1,55356,28271.0,16134.0,39485.0,34851.0,21674.0,65960.0,25.9,63779.0
2,01005,2000,29038,14909,14980,14058,36.0,10432,6036.0,4106.0,8610.0,6371.0,3915.0,10826.0,23.1,10612.0
3,01007,2000,20826,15928,10721,10105,35.1,7383,4439.0,2847.0,6347.0,4779.0,2414.0,8521.0,31.6,8400.0
4,01009,2000,51024,48098,25370,25654,36.3,19153,10927.0,6395.0,14930.0,12310.0,6462.0,23896.0,34.7,23333.0


In [102]:
control.describe()

,year,total_population,white,male,female,median_age,households,Age < 15,Age 15-24,Age 25-44,Age 45-64,Age >= 65,workforce,not_remote
count,45085.000000,4.508500e+04,4.508500e+04,4.508500e+04,4.508500e+04,45085.000000,4.508500e+04,4.508500e+04,4.508500e+04,4.508500e+04,4.508500e+04,4.508500e+04,4.508400e+04,4.508400e+04
mean,2014.858401,9.908666e+04,7.216429e+04,4.877636e+04,5.031030e+04,40.626969,4.169929e+04,1.911609e+04,1.359572e+04,2.655665e+04,2.549069e+04,1.432798e+04,4.569509e+04,4.315992e+04
std,5.474216,3.178993e+05,1.942428e+05,1.562064e+05,1.617340e+05,5.272344,1.230182e+05,6.268473e+04,4.411830e+04,9.323071e+04,7.880752e+04,4.131872e+04,1.492738e+05,1.400774e+05
min,2000.000000,4.100000e+01,1.400000e+01,2.300000e+01,1.500000e+01,18.000000,5.000000e+01,0.000000e+00,0.000000e+00,1.200000e+01,1.000000e+01,0.000000e+00,2.600000e+01,1.900000e+01
25%,2012.000000,1.122500e+04,9.095000e+03,5.631000e+03,5.557000e+03,37.400000,5.397000e+03,2.079000e+03,1.376000e+03,2.570000e+03,3.041000e+03,2.013000e+03,4.519000e+03,4.288000e+03
50%,2016.000000,2.593100e+04,2.180500e+04,1.291200e+04,1.301300e+04,40.600000,1.210700e+04,4.909000e+03,3.324000e+03,6.269000e+03,7.033000e+03,4.373000e+03,1.058000e+04,1.009600e+04
75%,2019.000000,6.622000e+04,5.623100e+04,3.277700e+04,3.327200e+04,43.600000,3.012900e+04,1.242500e+04,9.055464e+03,1.624600e+04,1.768300e+04,1.060700e+04,2.865100e+04,2.750250e+04
max,2022.000000,1.010572e+07,5.513981e+06,4.979641e+06,5.126081e+06,68.300000,3.599561e+06,2.266817e+06,1.499297e+06,3.125890e+06,2.554378e+06,1.415856e+06,4.811408e+06,4.542723e+06
